In [2]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

In [3]:
!!curl -O http://www.manythings.org/anki/fra-eng.zip

['  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current',
 '                                 Dload  Upload   Total   Spent    Left  Speed',
 '',
 '  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0',
 '  0     0    0     0    0     0      0      0 --:--:--  0:00:01 --:--:--     0',
 '  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0',
 '  0     0    0     0    0     0      0      0 --:--:--  0:00:03 --:--:--     0',
 '  0 5986k    0  2527    0     0    631      0  2:41:54  0:00:04  2:41:50   567',
 '  0 5986k    0 46307    0     0  11576      0  0:08:49  0:00:04  0:08:45  9349',
 ' 11 5986k   11  687k    0     0   137k      0  0:00:43  0:00:05  0:00:38  159k',
 ' 54 5986k   54 3268k    0     0   544k      0  0:00:10  0:00:06  0:00:04  757k',
 '100 5986k  100 5986k    0     0   855k      0  0:00:07  0:00:07 --:--:-- 1508k']

In [4]:
batch_size = 64  #this is the size for training
epochs = 100 #number of the iterations it will run for
latent_dim = 256  #latent dimensionality of the encoding space
num_samples = 10000 #Number of samples to train on
#Path to the data txt file
data_path = 'fra.txt'

In [6]:
#we are vectorizing this dara
input_texts = []
target_texts = []
input_characters = set()
target_characters = set()
with open(data_path, "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[: min(num_samples, len(lines) - 1)]:
    input_text, target_text, _ = line.split("\t")
    target_text = "\t" + target_text + "\n"
    input_texts.append(input_text)
    target_texts.append(target_text)
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)

In [10]:
input_characters

{' ',
 '!',
 '$',
 '%',
 '&',
 "'",
 ',',
 '-',
 '.',
 '0',
 '1',
 '2',
 '3',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 '?',
 'A',
 'B',
 'C',
 'D',
 'E',
 'F',
 'G',
 'H',
 'I',
 'J',
 'K',
 'L',
 'M',
 'N',
 'O',
 'P',
 'Q',
 'R',
 'S',
 'T',
 'U',
 'V',
 'W',
 'Y',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 'é'}

In [11]:
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])


In [16]:
num_decoder_tokens

94

In [17]:
print("Number of samples:", len(input_texts))
print("Number of unique input tokens:", num_encoder_tokens)
print("Number of unique output tokens:", num_decoder_tokens)
print("Max sequence length for inputs:", max_encoder_seq_length)
print("Max sequence length for outputs:", max_decoder_seq_length)

Number of samples: 10000
Number of unique input tokens: 71
Number of unique output tokens: 94
Max sequence length for inputs: 15
Max sequence length for outputs: 59


In [18]:
input_token_index = dict([(char,i) for i, char in enumerate(input_characters)])
target_token_index = dict([char,i] for i, char in enumerate(target_characters))

In [20]:
target_token_index

{'\t': 0,
 '\n': 1,
 ' ': 2,
 '!': 3,
 '$': 4,
 '%': 5,
 '&': 6,
 "'": 7,
 '(': 8,
 ')': 9,
 ',': 10,
 '-': 11,
 '.': 12,
 '0': 13,
 '1': 14,
 '2': 15,
 '3': 16,
 '5': 17,
 '8': 18,
 '9': 19,
 ':': 20,
 '?': 21,
 'A': 22,
 'B': 23,
 'C': 24,
 'D': 25,
 'E': 26,
 'F': 27,
 'G': 28,
 'H': 29,
 'I': 30,
 'J': 31,
 'K': 32,
 'L': 33,
 'M': 34,
 'N': 35,
 'O': 36,
 'P': 37,
 'Q': 38,
 'R': 39,
 'S': 40,
 'T': 41,
 'U': 42,
 'V': 43,
 'Y': 44,
 'a': 45,
 'b': 46,
 'c': 47,
 'd': 48,
 'e': 49,
 'f': 50,
 'g': 51,
 'h': 52,
 'i': 53,
 'j': 54,
 'k': 55,
 'l': 56,
 'm': 57,
 'n': 58,
 'o': 59,
 'p': 60,
 'q': 61,
 'r': 62,
 's': 63,
 't': 64,
 'u': 65,
 'v': 66,
 'w': 67,
 'x': 68,
 'y': 69,
 'z': 70,
 '\xa0': 71,
 '«': 72,
 '»': 73,
 'À': 74,
 'Ç': 75,
 'É': 76,
 'Ê': 77,
 'à': 78,
 'â': 79,
 'ç': 80,
 'è': 81,
 'é': 82,
 'ê': 83,
 'ë': 84,
 'î': 85,
 'ï': 86,
 'ô': 87,
 'ù': 88,
 'û': 89,
 'œ': 90,
 '\u2009': 91,
 '’': 92,
 '\u202f': 93}

In [29]:
encoder_input_data = np.zeros(
        (len(input_texts), max_encoder_seq_length, num_encoder_tokens), dtype='float32'
)
decoder_input_data = np.zeros(
            (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype='float32'
)
decoder_target_data = np.zeros(
            (len(input_texts), max_decoder_seq_length, num_decoder_tokens), dtype="float32"
)

In [30]:
for i, (input_text, target_text) in enumerate(zip(input_texts,target_texts)):
    for t, char in enumerate(input_text):
        encoder_input_data[i,t, input_token_index[char]] = 1.0
        encoder_input_data[i,t+1 :, input_token_index[" "]] = 1.0
    for t, char in enumerate(target_text):
        decoder_input_data[i,t, target_token_index[char]] = 1.0
        if t>0:
            decoder_target_data[i,t-1, target_token_index[char]] = 1.0
    decoder_input_data[i,t+1 :,target_token_index[" "]] = 1.0
    decoder_target_data[i,t:,target_token_index[" "]] = 1.0

In [34]:
encoder_input_data.shape

(10000, 15, 71)

In [35]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense

In [36]:
encoder_inputs = keras.Input(shape=(None, num_encoder_tokens))
encoder = keras.layers.LSTM(latent_dim, return_state = True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
encoder_states = [state_h, state_c]


In [39]:
decoder_inputs = keras.Input(shape=(None, num_decoder_tokens))
decoder_lstm = keras.layers.LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs, initial_state=encoder_states)
decoder_dense = keras.layers.Dense(num_decoder_tokens, activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)

In [ ]:
model = keras.Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
125/125 [==============================] - 7s 55ms/step - loss: 1.1282 - accuracy: 0.7265 - val_loss: 1.0471 - val_accuracy: 0.7245
Epoch 2/100
125/125 [==============================] - 6s 49ms/step - loss: 0.8309 - accuracy: 0.7758 - val_loss: 0.8178 - val_accuracy: 0.7680
Epoch 3/100
125/125 [==============================] - 6s 49ms/step - loss: 0.6783 - accuracy: 0.8096 - val_loss: 0.7077 - val_accuracy: 0.7952
Epoch 4/100
125/125 [==============================] - 6s 48ms/step - loss: 0.5930 - accuracy: 0.8279 - val_loss: 0.6589 - val_accuracy: 0.8087
Epoch 5/100
125/125 [==============================] - 6s 46ms/step - loss: 0.5441 - accuracy: 0.8412 - val_loss: 0.6256 - val_accuracy: 0.8155
Epoch 6/100
125/125 [==============================] - 6s 47ms/step - loss: 0.5064 - accuracy: 0.8520 - val_loss: 0.6011 - val_accuracy: 0.8227
Epoch 7/100
125/125 [==============================] - 6s 48ms/step - loss: 0.4779 - accuracy: 0.8593 - val_loss: 0.5703 - val_accuracy:

125/125 [==============================] - 6s 48ms/step - loss: 0.1048 - accuracy: 0.9669 - val_loss: 0.5780 - val_accuracy: 0.8723
Epoch 58/100
125/125 [==============================] - 6s 49ms/step - loss: 0.1026 - accuracy: 0.9679 - val_loss: 0.5842 - val_accuracy: 0.8719
Epoch 59/100
125/125 [==============================] - 6s 49ms/step - loss: 0.1003 - accuracy: 0.9683 - val_loss: 0.5879 - val_accuracy: 0.8721
Epoch 60/100
125/125 [==============================] - 6s 49ms/step - loss: 0.0980 - accuracy: 0.9687 - val_loss: 0.5929 - val_accuracy: 0.8721
Epoch 61/100
125/125 [==============================] - 6s 49ms/step - loss: 0.0956 - accuracy: 0.9696 - val_loss: 0.6020 - val_accuracy: 0.8715
Epoch 62/100
125/125 [==============================] - 6s 49ms/step - loss: 0.0943 - accuracy: 0.9701 - val_loss: 0.6056 - val_accuracy: 0.8719
Epoch 63/100
125/125 [==============================] - 6s 49ms/step - loss: 0.0923 - accuracy: 0.9706 - val_loss: 0.6062 - val_accuracy: 0.871